In [1]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from scipy.ndimage import zoom

In [2]:
# Load projections
path = r"model_uncertainty\normal"
proj_path = natsorted(glob(os.path.join(path, "*.npy")))
box_dim = LR.BOX_DIM
real_box_dim = 301

#load 3D model
path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(real_box_dim, path_real)
real_box = np.where(zoom(np.copy(real_box),1.4) >= 1, 1, 0)

# order projection based on custom selection (see report)
print(proj_path)

['model_uncertainty\\normal\\1_N-87_+27.npy', 'model_uncertainty\\normal\\2_N+00_+00.npy', 'model_uncertainty\\normal\\3_N+87_+27.npy', 'model_uncertainty\\normal\\4_N+87_-27.npy', 'model_uncertainty\\normal\\5_N-75_-22.npy', 'model_uncertainty\\normal\\6_N-43_-11.npy', 'model_uncertainty\\normal\\7_N-43_+13.npy', 'model_uncertainty\\normal\\8_N+43_-13.npy', 'model_uncertainty\\normal\\9_N+43_+13.npy']


In [3]:
box_total = np.zeros([445,445,445])
# select n projections to intersect in loop
for i in range(len(proj_path)): #len(proj_path)
  current_box = np.load(proj_path[i])
  box_total = box_total + current_box
box_total = np.where(box_total == len(proj_path), 1, 0)

#x,y,z = LR.boxToWorldCorrds(box_total,box_dim)

fx, fy, fz = D3.get3dBox(box_dim)

In [4]:
#mlab.points3d(x,y,z,
#                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0.5)
#mlab.points3d(x1,y1,z1,
#                  mode="cube", color=(0, 1, 0), scale_factor=1,opacity=0.5)
#mlab.points3d(fx,fy,fz,
#                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0)
#mlab.outline()
#mlab.show()

In [5]:
def dice_score(true, pred):
  true_bool = np.asarray(true).astype(np.bool)
  pred_bool = np.asarray(pred).astype(np.bool)
  inter = np.logical_and(true_bool,pred_bool)

  return (2* inter.sum()) / (true_bool.sum()
    + pred_bool.sum())

In [7]:
#add padding for reconstruction comparison
padding = int((box_dim - real_box.shape[0])/2)
print(padding)
real_box = np.pad(real_box, [(padding, padding), (padding, padding),(padding, padding)], mode='constant')

#x1,y1,z1 = LR.boxToWorldCorrds(real_box,real_box.shape[0])


12


In [8]:
#mlab.points3d(x,y,z,
#                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0.5)
#mlab.points3d(x1,y1,z1,
#                  mode="cube", color=(0, 1, 0), scale_factor=1,opacity=0.5)
#mlab.points3d(fx,fy,fz,
#                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0)
mlab.outline()
mlab.show()

In [19]:
#Calculate metrics for intersection of projections
np.set_printoptions(precision=5, suppress = True)
metrics= []
box_total = np.load(proj_path[0])
for i in range(len(proj_path)): #len(proj_path)
  #calculate box intersection
  current_box = np.load(proj_path[i])
  box_total = box_total + current_box
  box_total = np.where(box_total == 2, 1, 0)

  #calculate metrics for each reconstruction
  TN, FP, FN, TP = confusion_matrix(real_box.flatten(), box_total.flatten()).ravel()
  total = TN + FP + FN + TP
  accuracy = (TP + TN)/(FP+FN+TP+TN)
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  dice = dice_score(real_box.flatten(), box_total.flatten())
  #f1 = 2*(precision*recall)/(precision+recall)
  scores = np.array([TN/total, FP/total, FN/total, TP/total, dice, precision, recall])
  metrics.append(scores)
  print("Metrics using ", str(i+1), " projections: [tn, fp, fn, tp, dice, precision, recall]", scores)

Metrics using  0  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.86596 0.11644 0.00003 0.01757 0.23177 0.13111 0.99806 0.23177]
Metrics using  1  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.9491  0.0333  0.00003 0.01757 0.5132  0.3454  0.99806 0.5132 ]
Metrics using  2  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.97193 0.01046 0.0009  0.0167  0.74615 0.61482 0.94882 0.74615]
Metrics using  3  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.97355 0.00885 0.00091 0.0167  0.77401 0.65372 0.94856 0.77401]
Metrics using  4  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.97547 0.00693 0.00091 0.01669 0.80985 0.7067  0.94824 0.80985]
Metrics using  5  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.97753 0.00486 0.00092 0.01668 0.85222 0.7743  0.94758 0.85222]
Metrics using  6  projections: [tn, fp, fn, tp, dice, precision, recall, f1] [0.97815 0.00425 0.00093 0.01667 0.86565 0.79701 0.94721 0.86565]

: 